In [8]:
import torch
import triton
import triton.language as tl

In [ ]:
DEVICE = 'cuda:0'
torch_device = torch.device(DEVICE)

## Kernel to load every other element from a tensor

In [20]:
@triton.jit
def stride_copy_kernel(in_ptr, out_ptr, N, BLOCK_SIZE: tl.constexpr):
    pid = tl.program_id(0)
    offsets = pid * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)

    # Mask for input (every other element)
    input_mask = (offsets * 2) < N
    
    # Mask for output (contiguous elements)
    output_mask = offsets < (N // 2)

    inp_data = tl.load(in_ptr + (2 * offsets), mask=input_mask)
    alt_inp_data = tl.load(in_ptr + (2 * offsets + 1), mask=input_mask)
    tl.store(out_ptr + offsets, alt_inp_data, mask=output_mask)

In [21]:
# Wrapper function to launch the kernel
def stride_copy_wrapper(input_tensor, output_tensor):
    # Assume last dimension is the one to stride over
    n_elements = input_tensor.shape[-1]
    assert n_elements % 2 == 0, "Input tensor must have an even number of elements"
    
    BLOCK_SIZE = 64
    grid = (triton.cdiv(n_elements // 2, BLOCK_SIZE),)
    
    stride_copy_kernel[grid](
        input_tensor, 
        output_tensor, 
        n_elements, 
        BLOCK_SIZE
    )

In [22]:
# Create input tensor
input_tensor = torch.arange(300, dtype=torch.float32, device='cuda')
# Create output tensor to hold the result
output_tensor = torch.empty(150, dtype=torch.float32, device='cuda')

# Call the wrapper function
stride_copy_wrapper(input_tensor, output_tensor)

# Verify the result
print(output_tensor)  # Should contain [1, 3, 5, ..., 31]

tensor([  1.,   3.,   5.,   7.,   9.,  11.,  13.,  15.,  17.,  19.,  21.,  23.,
         25.,  27.,  29.,  31.,  33.,  35.,  37.,  39.,  41.,  43.,  45.,  47.,
         49.,  51.,  53.,  55.,  57.,  59.,  61.,  63.,  65.,  67.,  69.,  71.,
         73.,  75.,  77.,  79.,  81.,  83.,  85.,  87.,  89.,  91.,  93.,  95.,
         97.,  99., 101., 103., 105., 107., 109., 111., 113., 115., 117., 119.,
        121., 123., 125., 127., 129., 131., 133., 135., 137., 139., 141., 143.,
        145., 147., 149., 151., 153., 155., 157., 159., 161., 163., 165., 167.,
        169., 171., 173., 175., 177., 179., 181., 183., 185., 187., 189., 191.,
        193., 195., 197., 199., 201., 203., 205., 207., 209., 211., 213., 215.,
        217., 219., 221., 223., 225., 227., 229., 231., 233., 235., 237., 239.,
        241., 243., 245., 247., 249., 251., 253., 255., 257., 259., 261., 263.,
        265., 267., 269., 271., 273., 275., 277., 279., 281., 283., 285., 287.,
        289., 291., 293., 295., 297., 29

## RoPE Forward Pass Implementation

A work in progress implementation of RoPE forward pass in Triton.
Trying to understand the micro kernels required to make this efficient. 

In [77]:
torch.manual_seed(0)

# The d-head dimension
n_elements = 256

In [26]:
# Write the initial implementation for the RoPE kernel in Triton
import triton
import triton.language as tl
import torch

@triton.jit
def rope_kernel(
    q_ptr,
    k_ptr,
    cos_ptr,
    sin_ptr,
    out_ptr,
    n_elements,
    BLOCK_SIZE: tl.constexpr  # Block size for parallel processing
):
    # Define the program's index in the grid
    pid = tl.program_id(0)

    offsets = pid * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
    # Mask for input (every other element)
    input_mask = (offsets * 2) < n_elements
    # Mask for loading the sinusoid values, up to dmodel // 2
    sinusoid_mask = offsets < (n_elements // 2)
    
    q_real = tl.load(q_ptr + (2 * offsets), mask=input_mask)
    q_imag = tl.load(q_ptr + (2 * offsets + 1), mask=input_mask)
    k_real = tl.load(k_ptr + (2 * offsets), mask=input_mask)
    k_imag = tl.load(k_ptr + (2 * offsets + 1), mask=input_mask)

    cos = tl.load(cos_ptr + offsets, mask=sinusoid_mask)
    sin = tl.load(sin_ptr + offsets, mask=sinusoid_mask)

    q_rotated_real = q_real * cos - q_imag * sin
    q_rotated_imag = q_real * sin + q_imag * cos
    k_rotated_real = k_real * cos - k_imag * sin
    k_rotated_imag = k_real * sin + k_imag * cos

    # Store rotated vectors back to memory
    output_mask = offsets < n_elements
    tl.store(out_ptr + offsets * 2, q_rotated_real, mask=output_mask)
    tl.store(out_ptr + offsets * 2 + 1, q_rotated_imag, mask=output_mask)
    # tl.store(out_ptr + n_elements * 2 + offsets * 2, k_rotated_real, mask=output_mask)
    # tl.store(out_ptr + n_elements * 2 + offsets * 2 + 1, k_rotated_imag, mask=output_mask)


In [91]:
import torch.nn as nn
class RoPEEmbeddings(nn.Module):
    def __init__(self, dim, max_seq_len=4096, base=10000):
        super(RoPEEmbeddings, self).__init__()
        assert dim % 2 == 0, "dim must be even for RoPE."
        self.dim = dim
        self.max_seq_len = max_seq_len
        self.base = base

        self.register_buffer("inv_freq",
            1.0 / (self.base ** (torch.arange(0, dim, 2).float() / dim)), persistent=False)

        self.build_rope_cache()

    # Add the function to rotate half
    def rotate_half(self, x): 
        x1, x2 = x.chunk(2, dim=-1)
        return torch.cat((-x2, x1), dim=-1)
    
    def build_rope_cache(self):
        # Use this to precompute the RoPE cache
        pos_array = torch.arange(self.max_seq_len)
        theta = pos_array.unsqueeze(-1) * self.inv_freq
        cos = torch.cos(theta)
        sin = torch.sin(theta)
        #
        # Disable any interleaving or duplicating. CUDA doesn't need it.
        # cos = torch.cat([cos, cos], dim=-1)
        # sin = torch.cat([sin, sin], dim=-1)
        # instead do y[2i], y[2i+1] = x[i], x[i]
        # cos = torch.repeat_interleave(cos, repeats=2, dim=-1)
        # sin = torch.repeat_interleave(sin, repeats=2, dim=-1)

        self.register_buffer("cos", cos, persistent=False)
        self.register_buffer("sin", sin, persistent=False)

    def forward(self, x, positions=None):
        # Postitions is unused for now.
        out_real = x[..., ::2] * self.cos - x[..., 1::2] * self.sin
        out_imag = x[..., ::2] * self.sin + x[..., 1::2] * self.cos
        # Is there a better way to do this?
        out_interleaved = torch.cat((out_real.unsqueeze(-1),
                                    out_imag.unsqueeze(-1)), dim=-1)
        return out_interleaved.flatten(-2)


rope_ref = RoPEEmbeddings(n_elements, max_seq_len=1).to(DEVICE)

In [47]:
DEVICE = 'cuda:0' # There should be a better way to capture this
q_ptr = torch.randn((1, n_elements), device=DEVICE)
k_ptr = torch.randn((1, n_elements), device=DEVICE)
out_ptr = torch.zeros((1, n_elements), device=DEVICE)

torch_device = torch.device(DEVICE)
assert q_ptr.device == torch_device and k_ptr.device == torch_device and \
    out_ptr.device == torch_device

In [54]:
# Use a 1D grid.
grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )
rope_kernel[grid](
    q_ptr,
    k_ptr,
    rope_ref.cos,
    rope_ref.sin,
    out_ptr,
    n_elements,
    BLOCK_SIZE=32
)

In [92]:
import torch
from torchtune.modules import RotaryPositionalEmbeddings

# Initialize the RoPE module
rope = RotaryPositionalEmbeddings(dim=n_elements, max_seq_len=1).to(q_ptr.device)

# Apply RoPE to the input tensor
x_transformed = rope(q_ptr.view(1, 1, 1, n_elements))


In [96]:
# Comparing `torchtune` RopeEmbeddings with our implementation
# and Triton implementation.
print(torch.allclose(x_transformed, rope_ref(q_ptr)))
print(torch.allclose(x_transformed, out_ptr))

True
True


## Flash Attention

Following the tutorial from https://www.youtube.com/watch?v=zy8ChVd_oTM&t=7049s

In [1]:
# If we want to define a function that we can backpropagate through, the class needs to be
# derived from torch.autograd.Function.

class TritonAttention(torch.autograd.Function):
    @staticmethod
    def forward(ctx, Q, K, V, causal, softmax_scale):
        # ctx allows us to save intermediate results for backward pass.

        HEAD_DIM_Q, HEAD_DIM_K, HEAD_DIM_V = Q.shape[-1], K.shape[-1], V.shape[-1]

        BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_DIM = Q.shape

        O = torch.empty_like(Q)

        # What is stage?
        stage = 3 if causal else 1
        
        # Parallelize over the batch dimension and the number of heads
        grid = lambda args: (
            triton.cdiv(SEQ_LEN, args['BLOCK_SIZE_Q']),
            BATCH_SIZE * NUM_HEADS,
        )

In [94]:
import math

def test_op(BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_DIM, causal, dtype=torch.float16, DEVICE='cuda'):
    Q = (torch.empty(
            (BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_DIM), device=DEVICE, dtype=dtype
        )
        .normal_(mean=0.0, std=0.5)
    )
    K = (torch.empty(
            (BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_DIM), device=DEVICE, dtype=dtype
        )
        .normal_(mean=0.0, std=0.5)
    )
    V = (torch.empty(
            (BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_DIM), device=DEVICE, dtype=dtype
        )
        .normal_(mean=0.0, std=0.5)
    )

    softmax_scale = 1.0 / math.sqrt(HEAD_DIM)
    # Backpropagate through the output.
    dO = torch.randn_like(Q)

    mask = torch.tril(torch.ones(SEQ_LEN, SEQ_LEN), device=DEVICE)
    P = torch.matmul(Q, K.transpose(2, 3)) * softmax_scale
    
    if causal:
        P[..., mask == 0] = float('-inf')
    P = torch.softmax(P, dim=-1).half()
    ref_O = torch.matmul(P, V)
    ref_O.backward(dO)

    ref_dV, V.grad = V.grad.clone(), None
    ref_dP, P.grad = P.grad.clone(), None
    ref_dK, K.grad = K.grad.clone(), None
    ref_dQ, Q.grad = Q.grad.clone(), None

    # Compare with Triton implementation
    
    

True